In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.optimize as opt
from scipy.io import loadmat
import econtools 
import econtools.metrics as mt
import statsmodels.discrete.discrete_model as sm
import matplotlib.pyplot as plt


In [13]:
# ------------------------------------------------------------------------
# NOTE
# ------------------------------------------------------------------------
# Purpose: HW2 
# 
# Import data and stats
# 
#
# ------------------------------------------------------------------------

# Initialize file path

In [6]:
rootpath = '/Users/jingyuanwang/Dropbox/Course/ECON/IO/NU450/NU450_HW/450-1-HW2'
datapath = rootpath + '/' + 'data'
resultpath = rootpath + '/' + 'results'

# Parameters
Everything should be input in the following cell. 
All other cells are just operation and calculation.

In [54]:
# model parameters (true value)
betas = (5,1,1)
alpha = 1
sigma_alpha = 1
gammas = (2,1,1)

# import file names
matfiles = ['10markets3products', '100markets3products','100markets5products']
names = ['m10_prod3', 'm100_prod3', 'm100_prod5']

# Import Data

In [65]:
# 1 import
inputfiles = dict(zip(names, matfiles))
data = {}
for name, filename in inputfiles.items():
    file = datapath + '/Simulation Data/'+ filename
    data[name] = loadmat(file)


In [66]:
# 2. clean
name = 'm10_prod3'
df_product = (pd.DataFrame(np.concatenate( 
                          (data[name]['x1'],
                           data[name]['xi_all'],
                           data[name]['w'], 
                           data[name]['Z'],
                           data[name]['eta']), axis = 1 ))
              .rename(columns = {0:'x1',1:'x2',2:'x3',3:'xi', 4:'w', 5:'z',6:'eta'}) )
df_consumer = pd.DataFrame(data[name]['alphas']).rename(columns = {0:'alpha'})

# merge and expand data

In [ ]:
# a class, with 3 main dataset: product, consumer, production-consumer
# functions include statistics, simulate choice, and estimate BLP

# Note, for each output, save in product-consumer df and either product or consumer df as well 

# given product level data and consumer level data, get a product-consumer level long dataset

def merge_and_expand_to_long(df_product, df_consumer):
    
    return df



# Simulate consumer choice and welfare

In [ ]:
# Outline
# 1. simulate choice
# 2. compare with the given market share variable
# 3. save observables and final choice as a sample dataset. Make this a class
# 4. calculate welfare

In [ ]:
def simulate_consumer_choice(product_attribute_observed, product_attribute_unobs, price, taste, price_sensitivity):
    
    df['utility_ij'] = consumer_utility_each_prod() # apply for each row
    df.groupby['consumer_id'].max('utility_ij')
    
    return 1

def consumer_utility_each_prod( product_attribute_observed, product_attribute_unobs, price, taste, price_sensitivity):
    # this should be a scalar operation
    
    return 1


In [ ]:
def consumer_welfare():
    # scalar operation
    
    return 1


In [ ]:
def plot_estimated_real():
    return 1



# Estimation

In [ ]:
#1. answer questions

In [ ]:
#2. estimate delta and xi, compare with real data